In [9]:
import json
import requests
import time
import csv
import pandas as pd

In [10]:
headers = {
    'Host': 'stats.nba.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'x-nba-stats-origin': 'stats',
    'x-nba-stats-token': 'true',
    'Connection': 'keep-alive',
    'Origin': 'https://stats.nba.com',
    'Referer': 'https://stats.nba.com/',
    'Pragma': 'no-cache',
    'Cache-Control': 'no-cache'
}

In [11]:
#Würfe abrufen
franz_wuerfe =[]
seasons = ['2021-22']
playerid= "1630532"
teamid ="1610612753"
season_types = ['Regular+Season','Playoffs']
for season in seasons:
    for season_type in season_types:
        try:
            time.sleep(.6)
            season = season
            season_type = season_type
            url = 'https://stats.nba.com/stats/shotchartdetail??CFID=33&CFPARAMS={}&ContextFilter=&ContextMeasure=FGA&DateFrom=&DateTo=&GameID=&GameSegment=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerID={}&PlayerPosition=&PlusMinus=N&Position=&Rank=N&RookieYear=&Season={}&SeasonSegment=&SeasonType={}&TeamID={}&VsConference=&VsDivision=&mode=Advanced&showDetails=0&showShots=1&showZones=0'.format(
                        season, playerid, season, season_type, teamid)
            r = requests.get(url, headers=headers)
            data = r.json()
            relevant_data = data['resultSets'][0]
            rows = relevant_data['rowSet']
            franz_data = pd.DataFrame(rows)
            franz_data.columns = ['GRID_TYPE', 'GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING', 'SECONDS_REMAINING', 'EVENT_TYPE', 'ACTION_TYPE', 'SHOT_TYPE', 'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE', 'LOC_X', 'LOC_Y', 'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG', 'GAME_DATE', 'HTM', 'VTM']
            franz_data['SEASON'] = season
            franz_data['GAME_TYPE'] = season_type
            franz_data['GAME_TYPE'] = franz_data['GAME_TYPE'].replace(['Regular+Season','Playoffs'],['R','P'])
            franz_data["GAME_DATE"] = pd.to_datetime(franz_data["GAME_DATE"], format='%Y%m%d').dt.strftime('%d.%m.%Y')
            franz_data['MINUTES_RE']=franz_data['MINUTES_REMAINING'].astype(str)+':'+franz_data['SECONDS_REMAINING'].astype(str)
            franz_data["MINUTES_RE"] = pd.to_datetime(franz_data["MINUTES_RE"], format='%M:%S').dt.strftime('%M:%S')

            franz_data['SHOT_TYPE'] = franz_data['SHOT_TYPE'].replace(['2PT Field Goal','3PT Field Goal'],[2,3])
            franz_data = franz_data.drop(columns=['GRID_TYPE','PLAYER_ID','PLAYER_NAME','TEAM_ID','TEAM_NAME','EVENT_TYPE','MINUTES_REMAINING','SHOT_ZONE_BASIC','SHOT_ZONE_AREA','SHOT_ZONE_RANGE', 'SECONDS_REMAINING','SHOT_ATTEMPTED_FLAG'])
            franz_wuerfe.append(franz_data)
        except:
            pass
franz_wuerfe = pd.concat([franz_data for franz_data in franz_wuerfe])
franz_wuerfe.to_csv('franz.csv',index=False)

In [12]:

franz_geo = {"type": "FeatureCollection",
            "features": []
            }

with open('franz.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for shot in reader:
            # If there's a shot...
            if shot:
                # start conversions
                xfeet = int(shot['LOC_X'])
                yfeet = int(shot['LOC_Y'])

                x_km = xfeet / 95
                y_km = yfeet / 95

                x_geo = x_km
                y_geo = y_km * -1

                shot = {
                    "type": "Feature",
                    "geometry": {"type": "Point", "coordinates": [x_geo, y_geo]},
                    "properties": {
                        "gda": shot['GAME_DATE'],  # game date
                        "gid": int(shot['GAME_ID']),  # game id
                        "p": shot["PERIOD"],  # period
                        "r": shot['SHOT_MADE_FLAG'],  # shot result
                        "se_type": shot['GAME_TYPE'],
                        "sd": shot['SHOT_DISTANCE'],  # shot distance
                        "sr": shot['ACTION_TYPE'],  # shot range
                        "st": shot['SHOT_TYPE'],  # shot type
                        "y": shot['SEASON'],  # season/year
                        "h": shot['HTM'],  # opponent
                        "v": shot['VTM'],  # opponent
                        "id": shot['GAME_EVENT_ID']  # shot id
                    }
                }
                franz_geo["features"].append(shot)

with open('franz-shots.geojson', 'w+') as outfile:
        json.dump(franz_geo, outfile)